## Import the Required Libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
import contextily as ctx
import seaborn as sns
import geoplot as gplt
import geoplot.crs as gcrs
import folium
import matplotlib.ticker as ticker
import scipy.stats
import webbrowser
from scipy.stats import shapiro
import statsmodels.api as sm
from scipy.stats import spearmanr
from scipy.stats import kendalltau
from sklearn.preprocessing import MinMaxScaler
import matplotlib
from IPython.display import display, HTML, Image
import json
from scipy import stats
%matplotlib inline
import dataframe_image as dfi
import mgwr
import libpysal as ps
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import compare_surfaces, truncate_colormap
from matplotlib.colors import LinearSegmentedColormap
from shapely.geometry import Point
import pysal
from pysal.model import spreg
import textwrap
import matplotlib.colors as colors#
import statistics

## Import the Datasets

In [2]:
# PPFI version 2
file_path = "C:/Users/medaamm/OneDrive - University of Leeds/Project 1 Final Paper/DSDP-Project-1/Datasets/PPFI (V2).csv"
PPFI_V2 = pd.read_csv(file_path)

# LSOA 2011 to LSOA 2021
file_path = "C:/Users/medaamm/OneDrive - University of Leeds/Project 1 Final Paper/DSDP-Project-1/Datasets/LSOA (2011) to LSOA (2021).csv"
LSOA11_LSOA21 = pd.read_csv(file_path)

# Oxfordshire LSOAs 2011
file_path = "C:/Users/medaamm/OneDrive - University of Leeds/Project 1 Final Paper/DSDP-Project-1/Datasets/Oxfordshire LSOAs.csv"
Ox_LSOA11 = pd.read_csv(file_path)

## Merge the Datasets

In [3]:
PPFI_V2_Ox_Reordered = PPFI_V2.merge(LSOA11_LSOA21, on='LSOA21CD', how='inner')

In [4]:
PPFI_V2_Ox_Reordered = PPFI_V2_Ox_Reordered.merge(Ox_LSOA11[['LSOA11CD']], on='LSOA11CD', how='inner')

## Create the quantiles for the combined dimension

In [5]:
PPFI_V2_Ox_Reordered['pp_dec_Ox'] = pd.qcut(PPFI_V2_Ox_Reordered['combined'],
                                                 q=10,
                                                  labels=range(1,11))

## Create the quantiles for the separate PPFI dimensions

In [6]:
PPFI_V2_Ox_Reordered['SM_Prox_dec_Ox'] = pd.qcut(PPFI_V2_Ox_Reordered['domain_supermarket_proximity'],
                                             q=10, labels=range(1,11))
PPFI_V2_Ox_Reordered['SM_Acc_dec_Ox'] = pd.qcut(PPFI_V2_Ox_Reordered['domain_supermarket_accessibility'],
                                            q=10, labels=range(1,11))
PPFI_V2_Ox_Reordered['NonSM_Prox_dec_Ox'] = pd.qcut(PPFI_V2_Ox_Reordered['domain_nonsupermarket_proximity'],
                                            q=10, labels=range(1,11))
PPFI_V2_Ox_Reordered['fuelPov_dec_Ox'] = pd.qcut(PPFI_V2_Ox_Reordered['domain_fuel_poverty'],
                                             q=10, labels=range(1,11))
PPFI_V2_Ox_Reordered['SD_dec_Ox'] = pd.qcut(PPFI_V2_Ox_Reordered['domain_socio_demographic'],
                                            q=10, labels=range(1,11))
PPFI_V2_Ox_Reordered['FFF_dec_Ox'] = pd.qcut(PPFI_V2_Ox_Reordered['domain_food_for_families'],
                                             q=10, labels=range(1,11))

In [7]:
PPFI_V2_Ox_Reordered['E_Comm_dec_Ox'] = pd.cut(PPFI_V2_Ox_Reordered['domain_ecommerce_access'],bins=10, labels=range(1,11))

## Export to an Excel file

In [8]:
PPFI_V2_Ox_Reordered.to_excel("C:/Users/medaamm/OneDrive - University of Leeds/Project 1 Final Paper/DSDP-Project-1/Datasets/PPFI V2 Ox Reordered.xlsx")